In [2]:
from trino import dbapi
from trino.auth import OAuth2Authentication

conn = dbapi.connect(
    host="smart-cities-trino.pre-prod.cloud.vtti.vt.edu",
    port=443,
    http_scheme="https",
    auth=OAuth2Authentication(),   # <-- this is the right one
    catalog="smartcities_iceberg",  # optional default
    # schema="...",                # optional default
)


In [4]:

cur = conn.cursor()
cur.execute("SHOW SCHEMAS")
print(cur.fetchall())

Open the following URL in browser for the external authentication:
https://smart-cities-trino.pre-prod.cloud.vtti.vt.edu/oauth2/token/initiate/89141f2e2abc7ed1c8894caf1ea0a1a54e49fa7353a81e7b9dea8c4dfee74fb6
[['alexandria'], ['cci'], ['falls-church'], ['information_schema'], ['smart_cities_test'], ['system'], ['tables'], ['vtti']]


In [5]:
cur.execute("SHOW CATALOGS")
print(cur.fetchall())

[['smartcities_iceberg'], ['system']]


In [11]:
cur.execute("SHOW SCHEMAS FROM smartcities_iceberg")
print(cur.fetchall())

[['alexandria'], ['cci'], ['falls-church'], ['information_schema'], ['smart_cities_test'], ['system'], ['tables'], ['vtti']]


In [12]:
cur.execute("""
SELECT table_schema, table_name
FROM smartcities_iceberg.information_schema.tables
ORDER BY table_schema, table_name
LIMIT 20
""")
print(cur.fetchall())

[['alexandria', 'bsm'], ['alexandria', 'psm'], ['alexandria', 'safety-event'], ['alexandria', 'speed-distribution'], ['alexandria', 'vehicle-count'], ['alexandria', 'vru-count'], ['cci', 'bsm'], ['falls-church', 'hiresdata'], ['falls-church', 'maple_washington'], ['falls-church', 'mediantraveltimes'], ['falls-church', 'old_hiresdata'], ['falls-church', 'old_mediantraveltimes'], ['falls-church', 'old_priority_requests'], ['falls-church', 'old_safety_conflicts'], ['falls-church', 'old_safety_pedcompliance'], ['falls-church', 'old_safety_redlightrunners'], ['falls-church', 'old_safety_simpledelay'], ['falls-church', 'old_tmc'], ['falls-church', 'old_tmc_crosswalk'], ['falls-church', 'old_tmc_lanes']]


In [ ]:
import pandas as pd

df = pd.read_sql("""
SELECT intersection_id, avg(speed) AS avg_speed
FROM smartcities_iceberg.bsm_messages
WHERE timestamp >= current_date - interval '1' day
GROUP BY 1
ORDER BY avg_speed DESC
LIMIT 10
""", conn)
df.head()